In [69]:
import numpy as np
from datetime import date
import pandas as pd
from statistics import mode
from os import listdir
from os.path import isfile, join

def searchAtribute(atribute, string, isNumber = False):
    name = ""
    init = string.find(atribute) 

    while(string[init] != ' '):
        init += 1
    
    if not isNumber:
        init +=2
        limitCharacter = '"'
    else:
        limitCharacter ='\n'
        init +=1

    while(string[init] != limitCharacter):
        name += string[init]
        init += 1
    return name

def isPrivate(string): 
    result = string.find('"M17.5 7H17v-.25c0-2.76-2.24-5-5-5s-5 2.24-5 5V7h-.5C5.12 7 4 8.12 4 9.5v9C4 19.88 5.12 21 6.5 21h11c1.39 0 2.5-1.12 2.5-2.5v-9C20 8.12 18.89 7 17.5 7zM13 14.73V17h-2v-2.27c-.59-.34-1-.99-1-1.73 0-1.1.9-2 2-2 1.11 0 2 .9 2 2 0 .74-.4 1.39-1 1.73zM15 7H9v-.25c0-1.66 1.35-3 3-3 1.66 0 3 1.34 3 3V7z"')
    if(result > 0):
        return True
    return False

def getFollows(string):
    i = string.find('"Friends",\n   ')
    followsCount = ""

    while(string[i] != ':'):
        i += 1
    i += 2

    while(string[i] != '\n'):
        followsCount += string[i]
        i += 1
    return followsCount

def getTotalInteractions(string):
    i = string.find('"Tweets",\n   ')
    totalInteractions = ""

    while(string[i] != ':'):
        i += 1
    i += 2

    while(string[i] != '\n'):
        totalInteractions += string[i]
        i += 1
    return totalInteractions

def getDate(dateCrud):
    year = ""
    month = ""
    day = ""

    i = 0
    while(dateCrud[i] != '-'):
        year += dateCrud[i]
        i += 1

    i += 1
    while(dateCrud[i] != '-'):
        month += dateCrud[i]
        i += 1

    i += 1
    while(dateCrud[i] != 'T'):
        day += dateCrud[i]
        i += 1

    year = int(year)
    month = int(month)
    day = int(day)
    
    return date(year, month, day)

def getDayWithMorePublications(originalString):
    string = originalString
    
    init = string.find("datetime")
    if(init < 0):
        return "Ninguno"
    daysPublicateds = []

    while(init > 0):

        while(string[init] != "="):
            init += 1
        init += 2

        date = ""

        while(string[init] != 'T'):
            date += string[init]
            init+= 1

        dayOfWeek = pd.Timestamp(date).dayofweek
        daysPublicateds.append(dayOfWeek)
        
        newString = ""
        while(init != len(string)):
            newString += string[init]
            init += 1
        string = newString
        init = string.find("datetime")
    
    dayRecurrent = mode(daysPublicateds)

    if(dayRecurrent == 0):
        return "Lunes"
    elif(dayRecurrent == 1):
        return "Martes"
    elif(dayRecurrent == 2):
        return "Miércoles"
    elif(dayRecurrent == 3):
        return "Jueves"
    elif(dayRecurrent == 4):
        return "Viernes"
    elif(dayRecurrent == 5):
        return "Sábado"
    elif(dayRecurrent == 6):
        return "Domingo"

In [70]:
def getTwitterDataByProfile(path, folderPath, className):
    with open(folderPath + path, "r", encoding='utf-8-sig') as f:
        string = f.read()
    
    print(path)

    name = searchAtribute("additionalName", string)
    charactersNameCount = len(name)

    imageUrl = searchAtribute("contentUrl", string)
    haveImage = not ("default_profile_images" in imageUrl)
    haveImage = "Sí" if (haveImage) else "No"

    followersCount = searchAtribute('userInteractionCount', string, isNumber=True)
    followsCount   = getFollows(string)

    isPrivateProfile =isPrivate(string)
    isPrivateProfile = "Sí" if (isPrivateProfile) else "No"

    dateCreated = searchAtribute("dateCreated", string)
    totalInteractions = getTotalInteractions(string)

    startDate = getDate(dateCreated)
    today = date.today()
    today = str(today) + "T"
    endDate = getDate(today)

    totalDays = endDate - startDate

    meanPublications = 0
    if(totalDays):
        meanPublications = int(totalInteractions) / totalDays.days

    isComented = "Sí"
    if(not int(totalInteractions)):
        isComented = "No"

    dayWhitMorePublications = getDayWithMorePublications(string)

    #print(f'El nombre de usuario es {name}')
    #print(f'Cantidad de caracteres en su nombre: {charactersNameCount}')
    #print(f'¿Tiene foto de perfil? {haveImage}')
    #print(f'Cantidad de seguidores: {followersCount}')
    #print(f'Cantidad de seguidos: {followsCount}')
    #print(f'Promedio de publicaciones por día: {meanPublications}')
    #print(f'¿Es perfil privado? {isPrivateProfile}')
    #print(f'Día que más comenta {dayWhitMorePublications}')
    #print(f'¿Comenta publicaciones? {isComented}\n')

    data = []
    data.append(name)
    data.append(charactersNameCount)
    data.append(haveImage)
    data.append(followersCount)
    data.append(followsCount)
    data.append(meanPublications)
    data.append(isPrivateProfile)
    data.append(dayWhitMorePublications)
    data.append(isComented)
    data.append(className)

    return data

In [71]:
myPath = "./realProfiles/"
onlyfiles = [f for f in listdir(myPath) if isfile(join(myPath, f))]

myPath2 = "./fakeProfiles/"
onlyfiles2 = [f for f in listdir(myPath2) if isfile(join(myPath2, f))]



In [72]:
import csv
header = ['nombre', 'cantidad_caracteres_nombre', 
          'tiene_foto_perfil', 
          'cantidad_seguidores', 
          'cantidad_seguidos', 
          'promedio_publicaciones', 
          'perfil_privado',
          'dia_mas_publica',
          'comenta_publicaciones',
          'clase']

with open('twitterData.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)

    for path in onlyfiles2:
        data = getTwitterDataByProfile(path, myPath2, className="Fake")
        writer.writerow(data)

    for path in onlyfiles:
        data = getTwitterDataByProfile(path, myPath, className="Real")
        writer.writerow(data)
    f.close()

print("It's done")


(1) Catalina Queen (@Catalin31937081) _ Twitter.html
Jade Crespo (@JadeCrespo8) _ Twitter.html
AraV12 (@V12Ara) _ Twitter.html
Brianna Luna (@BriannaLun_) _ Twitter.html
Isolda Mancillas (@IsoldaMancillas) _ Twitter.html
(1) x0c4l0r0104 j4w64308 (@bp252475057g) _ Twitter.html
(1) Javier Esparza (@JavierE16716229) _ Twitter.html
Betsabe Aguirre (@BetsabeAguirre_) _ Twitter.html
(1) (_◠_‿_◕_)จ๊อบ (@YuththnaPhumi) _ Twitter.html
Martha Pablo_pocket (@MarthaPablo8) _ Twitter.html
(1) Peer Adej (@AdejPeer) _ Twitter.html
plácido García (@PlaciddoG__) _ Twitter.html
(1) ไม่ต้องพู๊ด (@Flooky124) _ Twitter.html
(1) ณัฐวัฒน์ วงค์จันทา (@Ka25402540) _ Twitter.html
Denise Pastor (@Pastor10Denise) _ Twitter.html
(1) Camila (@Camila76800538) _ Twitter.html
(1) Tweets con contenido multimedia de Ángeles Franco (@AngelesFra_) _ Twitter.html
(1) จู๋ เล็ก!!! (@TNrCC4VoQpOGSWy) _ Twitter.html
(1) pim120430 (@pim120430198853) _ Twitter.html
Anahi Bravo (@anahibravoab) _ Twitter.html
Jannet Conde (@Conde